In [74]:
import ast
import subprocess
import numpy as np
import time
import sys

In [49]:
# Read files for list of all patient ids and list of kmers
with open("data/patient_list.txt") as f:
    patient_list = f.read().split()

with open('data/1000_most_common_kmers.py', 'r') as f:
    most_common = ast.literal_eval(f.read())
temp = []
for kmc in most_common:
    temp.append(kmc[0])
kms_str = str(temp).replace(',', "").replace("'", '')[1:-1]

In [50]:
start_i = 0
kmer_table = np.zeros((len(patient_list), 1000))

In [65]:
# Query all patients for specific kmers, but store in nparray
def query_np(patients, kms):
    global kmer_table
    global start_i
    i = start_i
    n = 0 # kill after 20 iterations
    if i != 0:
        print(f"Restarting at patient #{i}")
    buffer = np.zeros((5, 1000))
    for p in patients:
        jf = f"/u/leucegene/data/rnaseq/jellyfish/EXT001/{p}/2.2.3/kmers.k31.CL2.jf"
        cmd = f"jellyfish query {jf} {kms}"
        output = subprocess.check_output(cmd, shell=True).decode("utf-8")
        j = 0
        for line in output.splitlines():
            km, cnt = line.split()
            buffer[i%5,j] = int(cnt)
            j += 1
        i += 1
        if i % 1 == 0:
            timestamp = time.strftime("%H:%M:%S", time.localtime())
            print(f"{timestamp}: Queried patient #{i}...")
        if i % 5 == 0:
            # Dump buffer
            for k in range(5):
                kmer_table[i-k-1,:] = buffer[-k-1,:]
            buffer = np.zeros((5, 1000))
            print(f"Dumped buffer after patient {i}!")
            start_i = i
        n += 1
        if n >= 20:
            return

In [67]:
# For each patient, extract kmer counts and put it in a nparray
while start_i < 260:
    query_np(patient_list, kms_str)
print("Done querying!")

Done querying!


In [80]:
# Save kmer table to file
print(kmer_table.shape)
print(f"{sys.getsizeof(kmer_table) / 1e6:.3} MB")
np.save("data/kmer_table", kmer_table)

(260, 1000)
2.08 MB


In [87]:
# Loading kmer table from file
test = np.load("data/kmer_table.npy")
print(test.shape)
print(f"{sys.getsizeof(test) / 1e6:.3} MB")

(260, 1000)
2.08 MB
